# Navier-Stokes simulation in 2D
The simulation of a pressure driven lid and a pipe is possible. In these geometries obstuctions can be added.

In [ ]:
#%matplotlib notebook
import time
import numpy as np

from types import SimpleNamespace

from functions import *
from constants import *
from simulation import *

start_time = time.time()

# Initial conditions 
const = SimpleNamespace()
const.Re = 1e1            # Reynolds number
const.dt = 1e-4           # time step
const.tf = 1e-1            # final time
const.lx = 2              # width of box
const.ly = 1              # height of box
const.cholesky = True
const.velocity_arr = np.linspace(1, 1.2, 1)
print('Velocities =', const.velocity_arr)

# Grid multiplier options
multi_arr = np.arange(20, 21, 1)
print('Multipliers =', multi_arr)

# Naca angles 
angle_arr = np.arange(30, 61, 3) # in degrees
print('Angles =', angle_arr)

# Data Processing settings
const.nsteps = 200      # step interval for graphic output
const.rho_arrow = 20    # Density of arrows in quiver plot per unit length l
const.plot_P = True
const.plot_U = True
const.plot_Q = False

const.save_fig = False
const.fig_dir ='./movie_images/' 
const.fig_id = 'th' 

results = SimpleNamespace()
results.obj_F = np.zeros((len(angle_arr), len(multi_arr), len(const.velocity_arr)), dtype=float)
results.angle = np.zeros(len(angle_arr))
results.grid_multi = np.zeros(len(multi_arr))
results.v = np.zeros(len(const.velocity_arr))
results.kin_energy = np.zeros((len(angle_arr), len(multi_arr), len(const.velocity_arr), int(const.tf/const.dt)))

for j, multiplier in enumerate(multi_arr):
    const.nx = 60*multiplier  # number of x-gridpoints
    const.ny = 30*multiplier  # number of y-gridpoints
    
    const.fig_id = str(np.round(const.nx)) + '_' + str(np.round(const.ny)) + '_' + const.fig_id
    
    ## Set other constants (incl. grid) which are determined by initial conditions specified above
    const = constants(const)

    # IC's
    data = SimpleNamespace()
    data = initialisation(const, data)

    # BC's
    ## North, East, South, West
    bc = SimpleNamespace()
    boundary_type = 'horizontal_sea'

    # Object 
    obj = SimpleNamespace(sort = 'naca') #'hemicircle')
    obj.profile = '2412' # options ['0009', '0012', '2412', 2414', '2415', '6409' , '0006', '0008', '0010', '0012', '0015']
    
    for k, obj.angle in enumerate(angle_arr):
        obj = creat_obj(const, obj)

        # Operator matrices on P, U, V, Q respectively
        LP = SimpleNamespace()
        LP = laplacian(const, LP)
        LP = laplace_obj(const, LP, obj)
        if const.cholesky:
            LP = cholesky_decomposition(LP)

        init_time = time.time()
        print('Init time', np.round(init_time - start_time,2), 's')

        # Simulation
        for i, const.velocity in enumerate(const.velocity_arr):
            print('Angle =', obj.angle)
            print('Multiplier =', multiplier)
            print('Velocity =', const.velocity)
            
            # BC's
            bc = set_BC(const, bc, boundary_type)

            # Boundary matrices
            bc = set_BM(const, bc)

            data = simulation(const, bc, obj, LP, data)

            results.obj_F[k, j, i] = data.obj_F

            results.angle[k] = obj.angle*1
            results.grid_multi[j] = multiplier*1
            results.v[i] = const.velocity*1
            results.kin_energy[k, j, i, :] = data.kin_energy
    
end_time = time.time()
print('Runtime', np.round(end_time - start_time,2), 's')

#np.savez('./results/2412_dte-4_angle_vs_F_2', F = results.obj_F, angle = results.angle, multiplier = results.grid_multi, v = results.v, E_kin = results.kin_energy)

# Energy check

In [ ]:
#%matplotlib notebook
plt.plot(data.kin_energy)
plt.show()

## Other Visualisation

In [ ]:
# Visual representation of the boundary of the object
plt.plot(obj.lbound[0], obj.lbound[1], 'k.')
plt.plot(obj.rbound[0], obj.rbound[1], 'g.')
plt.plot(obj.tbound[0], obj.tbound[1], 'r.')
plt.plot(obj.bbound[0], obj.bbound[1], 'b.')
plt.title('Left/right/top/bottom (k, g, r, b)')
plt.show()

In [ ]:
# Plots of the Force vs the angle and the initial velocity
plt.plot(results.angle, results.obj_F[:,0,0], 'x')
plt.show()
plt.plot(results.v, results.obj_F[0,0,:], 'x')
plt.show()

In [ ]:
# Visualisation of the grid

import numpy as np
import matplotlib.pyplot as plt

x0 = 0
xL = const.nx

y0 = 0
yL = const.ny

Obj = obj.Pgrid

u_x = np.arange(x0+1, xL, 1)
u_y = np.arange(y0+0.5, yL+0.5, 1)
[X_u, Y_u] = np.meshgrid(u_x, u_y)

v_x = np.arange(x0+0.5, xL+0.5, 1)
v_y = np.arange(y0+1, yL, 1)
[X_v, Y_v] = np.meshgrid(v_x, v_y)

p_x = np.arange(x0+0.5, xL + 0.5, 1)
p_y = np.arange(y0+0.5, yL + 0.5, 1)
[X_p, Y_p] = np.meshgrid(p_x, p_y)

my_dpi = 96
plt.figure(figsize=(xL*40/my_dpi, yL*40/my_dpi), dpi=my_dpi)
plt.plot(np.reshape(X_u, (-1,)), np.reshape(Y_u, (-1,)), 'x', label='U')
plt.plot(np.reshape(X_v, (-1,)), np.reshape(Y_v, (-1,)), '+', label='V')
plt.plot(np.reshape(X_p, (-1,)), np.reshape(Y_p, (-1,)), 'o', label='P')

plt.plot(obj.coord_P[0] + 0.5, obj.coord_P[1] + 0.5, 'or')
plt.plot(obj.coord_U[0] + 1, obj.coord_U[1] + 0.5, 'xr')
plt.plot(obj.coord_V[0] + 0.5, obj.coord_V[1] + 1, '+r')
plt.plot(obj.coord_Q[0] + 1, obj.coord_Q[1] + 1, 'ob')

plt.legend()
plt.imshow(Obj.T, extent=[x0, xL, y0, yL])
plt.tight_layout()

plt.show()